In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
'''os.chdir('/content')
!mkdir /root/.kaggle
!echo '{"username":"daniel1998xx","key":"c99226b7cd73b12d4681a78a5b02d227"}' > /root/.kaggle/kaggle.json
!kaggle datasets download -d shuyangli94/interview-npr-media-dialog-transcripts
if not os.path.exists("/content/NPR_Data"):
    os.makedirs("/content/NPR_Data")
'''

'os.chdir(\'/content\')\n!mkdir /root/.kaggle\n!echo \'{"username":"daniel1998xx","key":"c99226b7cd73b12d4681a78a5b02d227"}\' > /root/.kaggle/kaggle.json\n!kaggle datasets download -d shuyangli94/interview-npr-media-dialog-transcripts\nif not os.path.exists("/content/NPR_Data"):\n    os.makedirs("/content/NPR_Data")\n'

In [3]:
'''!unzip interview-npr-media-dialog-transcripts.zip -d NPR_Data'''

'!unzip interview-npr-media-dialog-transcripts.zip -d NPR_Data'

In [4]:
'''os.chdir('/content/NPR_Data')'''

"os.chdir('/content/NPR_Data')"

In [5]:
df = pd.read_csv('utterances.csv')
df['speaker'] = df['speaker'].str.upper()
df['speaker'] = df['speaker'].str.split(', H', expand=True)[0]

In [6]:
df = df[df['speaker'] != '_NO_SPEAKER']

In [7]:
df['utterance_len'] = df['utterance'].str.len()

In [8]:
df = df.dropna()
host_id_df = pd.read_json('host_id.json', orient='index')

In [9]:
host_id_df = pd.read_json('host_id.json', orient='index')
host_id_df.reset_index(inplace=True)
host_id_df = host_id_df.rename(columns = {'index':'speaker'})

In [10]:
host_id_df['speaker'] = host_id_df['speaker'].astype('string')
host_id_df['speaker'] = host_id_df['speaker'].str.upper()
host_id_df.rename(columns={0:'host_id'}, inplace=True)

In [11]:
mergedf = df.merge(host_id_df, how='left', on='speaker')

In [12]:
mergedf['host_id'] = mergedf['host_id'].fillna(-1)

In [13]:
PreProcess_df = mergedf[mergedf['host_id'] != -1]

In [14]:
PreProcess_df = PreProcess_df.loc[PreProcess_df['utterance_len'] >= 100]

In [15]:
top_speakers = PreProcess_df.groupby(['speaker']).size().loc[PreProcess_df.groupby(['speaker']).size() > 20000]
df = pd.DataFrame(PreProcess_df.loc[PreProcess_df['speaker'].isin(top_speakers.index.values)])
df = df.reset_index(drop=True)

In [16]:
df['utterance'] = df['utterance'].str.lower()

In [17]:
df['unidentified'] = df['utterance'].str.startswith('unidentified')
df = df[df['unidentified'] == False]

In [18]:
df.count()

episode          289355
episode_order    289355
speaker          289355
utterance        289355
utterance_len    289355
host_id          289355
unidentified     289355
dtype: int64

In [19]:
df['speaker'].value_counts()

NEAL CONAN        126971
STEVE INSKEEP      33787
ROBERT SIEGEL      29948
IRA FLATOW         29888
FARAI CHIDEYA      23937
MELISSA BLOCK      23134
RENEE MONTAGNE     21690
Name: speaker, dtype: int64

In [20]:
df = df.groupby('speaker').apply(lambda x: x.sample(34000) if x['speaker'].iloc[0] == 'NEAL CONAN' else x)
df['speaker'].value_counts()

NEAL CONAN        34000
STEVE INSKEEP     33787
ROBERT SIEGEL     29948
IRA FLATOW        29888
FARAI CHIDEYA     23937
MELISSA BLOCK     23134
RENEE MONTAGNE    21690
Name: speaker, dtype: int64

In [21]:
nltk.download('stopwords') 
nltk.download('wordnet')
nltk.download('omw-1.4')

NameError: name 'nltk' is not defined

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

In [ ]:
def preprocess_text(text):
    tokeniser = RegexpTokenizer(r'\w+')
    tokens = tokeniser.tokenize(text)

    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]

    keywords= [lemma for lemma in lemmas if lemma not in stopwords.words('english')]
    return keywords

In [ ]:
utterance_list = df['utterance'].to_list()
final_list = []
complete_list = []
for utterance in utterance_list:
    processed = preprocess_text(utterance)
    final_list.append(processed)
complete_list.extend(final_list)
df['processed_utterance'] = complete_list

In [ ]:
df['processed_utterance'] = df['processed_utterance'].apply(lambda x: ' '.join(x))

In [ ]:
df.to_pickle("processed_data.pkl")

In [ ]:
df = pd.read_pickle("processed_data.pkl")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
#from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
X = df.processed_utterance
y = df.speaker
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
vector = TfidfVectorizer(analyzer='word', max_features = 850, ngram_range=(1, 1), 
                       binary=False, norm=None, smooth_idf=True, strip_accents=None,
                       sublinear_tf=True, use_idf=False)

df_transformed = vector.fit_transform(X_train)

In [ ]:
multi_nb_model = MultinomialNB()
multi_nb_model.fit(df_transformed, y_train)
print ("Model accuracy within dataset: ", multi_nb_model.score(df_transformed, y_train))

In [ ]:
print(classification_report(cross_valmodel_pred_pd["label"], cross_valmodel_pred_pd["prediction"]))
print(accuracy_score(cross_valmodel_pred_pd["label"], cross_valmodel_pred_pd["prediction"]))

In [ ]:
test_text = ["So for a top competitor like Lance to try to"]
test_text_transform = vector.transform(test_text)

print (multi_nb_model.predict(test_text_transform)," most likely said it.")

In [ ]:
pipe = make_pipeline(TfidfVectorizer(), MultinomialNB())
pipe.steps

param_grid = {}
param_grid["tfidfvectorizer__max_features"] = [500, 1000, 15000]
param_grid["tfidfvectorizer__ngram_range"] = [(1,1), (1,2), (2,2)]
#param_grid["tfidfvectorizer__stop_words"] = ["english", None]
param_grid["tfidfvectorizer__strip_accents"] = ["ascii", "unicode", None]
param_grid["tfidfvectorizer__analyzer"] = ["word", "char"]
param_grid["tfidfvectorizer__binary"] = [True, False]
param_grid["tfidfvectorizer__norm"] = ["l1", "l2", None]
param_grid["tfidfvectorizer__use_idf"] = [True, False]
param_grid["tfidfvectorizer__smooth_idf"] = [True, False]
param_grid["tfidfvectorizer__sublinear_tf"] = [True, False]


grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', verbose = 3, n_jobs = -1)
grid.get_params().keys()

In [ ]:
grid.fit(X_train,y_train)
print(grid.best_params_)
print(grid.best_score_)